### Libraries

In [1]:
'generic imports'
import os
import pandas as pd
import sys
sys.path.append(os.path.abspath('..'))
from src import utils
import importlib
importlib.reload(utils)
import datetime

'machine learning library imports'
from sklearn import tree
from sklearn import metrics
from sklearn.utils import shuffle

### Load data

In [2]:
AUGMENTATION = 'SMOTE-NC'

data_dir = os.path.abspath('../data')

# Load the train and test datasets
df_train, df_test = utils.load_dataset(data_directory=data_dir, 
                                       augmentation=AUGMENTATION, 
                                       ignore_columns=['mbtcp.unit_id', 
                                                       'mbtcp.trans_id'])                                    

Loading complete.
Train data: 1500000 rows, 46 columns. 
Test data: 381934 rows, 46 columns.


### Data preparation

In [3]:
# Creates X_train, y_train
X_train = df_train.drop(['Attack_label', 'Attack_type'], axis=1)
y_train = df_train['Attack_type']

# Creates X_test, y_test
X_test = df_test.drop(['Attack_type', 'Attack_label'], axis=1)
y_test = df_test['Attack_type']

#### Convert categorical features to one-hot encoded features

In [5]:
# Encode the training and test labels if needed
X_train_enc, X_test_enc, info = utils.encode_categorical(X_train, X_test)

Categorical features to be encoded:

http.referer
mqtt.protoname
http.request.version
mqtt.conack.flags
mqtt.topic
http.request.method
dns.qry.name.len

Encoding complete.
No of features before encoding: 44
No of features after encoding: 83


#### Label encoding

In [6]:
y_train_enc, y_test_enc, le = utils.encode_labels(y_train, y_test)

Attack_type and encoded labels:

Backdoor                0
DDoS_HTTP               1
DDoS_ICMP               2
DDoS_TCP                3
DDoS_UDP                4
Fingerprinting          5
MITM                    6
Normal                  7
Password                8
Port_Scanning           9
Ransomware              10
SQL_injection           11
Uploading               12
Vulnerability_scanner   13
XSS                     14


### Model Training

In [7]:
# Shuffle training data
X_train_enc, y_train_enc = shuffle(X_train_enc, y_train_enc, random_state=42)

# Instantiate the decision tree classifier
dt = tree.DecisionTreeClassifier(random_state=42)

# Train the model
dt_clf = dt.fit(X_train_enc, y_train_enc)

### Model Evaluation

In [8]:
predictions = dt_clf.predict(X_test_enc)
predictions[:3]

array([ 7, 11,  4])

In [9]:
# Calculate metrics
accuracy = metrics.accuracy_score(y_test_enc, predictions)
precision_w = metrics.precision_score(y_test_enc, predictions, average='weighted', zero_division=1)
recall_w = metrics.recall_score(y_test_enc, predictions, average='weighted')
f1_score_w = metrics.f1_score(y_test_enc, predictions, average='weighted')
precision_m = metrics.precision_score(y_test_enc, predictions, average='macro', zero_division=1)
recall_m = metrics.recall_score(y_test_enc, predictions, average='macro')
f1_score_m = metrics.f1_score(y_test_enc, predictions, average='macro')


In [10]:
# Create dictionary for results
results = {
    "model": "Decision Tree",
    "augmentations": AUGMENTATION,
    "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "accuracy": accuracy,
    "precision_macro": precision_m,
    "recall_macro": recall_m,
    "f1_macro": f1_score_m,
    "precision_weighted": precision_w,
    "recall_weighted": recall_w,
    "f1_weighted": f1_score_w
    }

utils.print_results_table(results)

╒══════════════════════╤═════════╕
│ Metric               │ Value   │
╞══════════════════════╪═════════╡
│ Accuracy             │ 97.95%  │
├──────────────────────┼─────────┤
│ Precision (macro)    │ 87.36%  │
├──────────────────────┼─────────┤
│ Recall (macro)       │ 91.16%  │
├──────────────────────┼─────────┤
│ F1 (macro)           │ 88.16%  │
├──────────────────────┼─────────┤
│ Precision (weighted) │ 98.06%  │
├──────────────────────┼─────────┤
│ Recall (weighted)    │ 97.95%  │
├──────────────────────┼─────────┤
│ F1 (weighted)        │ 97.99%  │
╘══════════════════════╧═════════╛


#### Save Metrics Results 

In [11]:
# save results to csv   
utils.save_results_to_csv([results], '../results/metrics/dt.csv')

#### Confusion Matrix

In [12]:
conf_mat = metrics.confusion_matrix(y_test_enc, predictions)

attack_labels = ['Backdoor', 'DDoS_HTTP', 'DDoS_ICMP', 'DDoS_TCP', 'DDoS_UDP', 
'Fingerprinting', 'MITM', 'Normal', 'Password', 'Port_Scanning', 'Ransomware', 
'SQL_injection', 'Uploading', 'Vulnerability_scanner', 'XSS']

# Create a dataframe from the confusion matrix
conf_mat_df = pd.DataFrame(conf_mat, 
                            index = attack_labels, 
                            columns = attack_labels)
conf_mat_df.index.name = 'Actual'
conf_mat_df.columns.name = 'Predicted'


# Save the confusion matrix
conf_mat_df.to_csv(f"../results/conf_matrix/{results['model']}_{results['augmentations']}.csv")
conf_mat_df

Predicted,Backdoor,DDoS_HTTP,DDoS_ICMP,DDoS_TCP,DDoS_UDP,Fingerprinting,MITM,Normal,Password,Port_Scanning,Ransomware,SQL_injection,Uploading,Vulnerability_scanner,XSS
Actual,,,,,,,,,,,,,,,
Backdoor,4522,0,0,0,0,108,0,0,0,62,90,0,0,0,0
DDoS_HTTP,0,8423,0,0,0,0,0,0,0,0,0,0,0,283,922
DDoS_ICMP,0,0,13469,0,0,32,0,0,0,0,0,0,0,0,0
DDoS_TCP,0,0,0,10009,0,0,0,0,0,0,0,0,0,0,0
DDoS_UDP,0,0,0,0,24601,0,0,0,0,0,0,0,0,0,0
Fingerprinting,3,0,0,0,0,112,0,0,0,22,9,0,0,0,0
MITM,0,0,0,0,0,0,76,0,0,0,0,0,0,0,0
Normal,0,0,0,0,0,0,0,272776,0,0,0,0,0,0,0
Password,0,5,0,0,0,0,0,0,8346,0,0,970,785,0,2
